```markdown
# In this notebook we will clean the raw datasets we have for migrants and global crypto adoption index

We first need to import the required libraries.
```

In [4]:
import pandas as pd
import numpy as np
!pip install openpyxl
from openpyxl import load_workbook


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing the Global Crypto Adoption Dataset

We begin by importing the **Global Crypto Adoption** dataset.

📂 **Source**: [Global Crypto Adoption Dataset](https://github.com/MIT-Emerging-Talent/ET6-CDSP-group-12-repo/tree/main/1_datasets/raw_datasets_files)

The dataset contains information on cryptocurrency adoption rates across various countries and will be used for further analysis and visualization.

We'll first load the dataset into a DataFrame using `pandas` and then inspect its structure.


In [17]:
# Importing the Global Crypto Adoption Dataset

# Link to raw Excel file
url = "https://raw.githubusercontent.com/MIT-Emerging-Talent/ET6-CDSP-group-12-repo/data_cleaning/1_datasets/raw_datasets_files/The_Global_crypto_Adoption_Index_2022_to_2024.xlsx"

# Sheet names (as you said)
GCAI_2022 = "The 2022 Global crypto Adoption"
GCAI_2023 = "The 2023 Global crypto Adoption"
GCAI_2024 = "The 2024 Global crypto Adoption"

# Read each sheet
df_2022 = pd.read_excel(url, sheet_name=GCAI_2022)
df_2023 = pd.read_excel(url, sheet_name=GCAI_2023)
df_2024 = pd.read_excel(url, sheet_name=GCAI_2024)


# Optional: View the first few rows
print(df_2024.head())


   Rank        Country                                      Region  \
0     1          India  Central & Southern Asia and Oceania (CSAO)   
1     2        Nigeria                          Sub-Saharan Africa   
2     3      Indonesia  Central & Southern Asia and Oceania (CSAO)   
3     4  United States                               North America   
4     5        Vietnam  Central & Southern Asia and Oceania (CSAO)   

   Overall Index Ranking  Centralized Service Value Received Ranking  \
0                      1                                           1   
1                      2                                           5   
2                      3                                           6   
3                      4                                           2   
4                      5                                           3   

   Retail Centralized Service Value Received Ranking  \
0                                                  1   
1                                 

In [23]:
# Keep only Rank, Country, Region, and add Year
df_2022 = df_2022[["Rank", "Country"]]
df_2022["Year"] = 2022

df_2023 = df_2023[["Rank", "Country", "Region"]]
df_2023["Year"] = 2023

df_2024 = df_2024[["Rank", "Country", "Region"]]
df_2024["Year"] = 2024

df_2024.head()

,Rank,Country,Region,Year
0,1,India,Central & Southern Asia and Oceania (CSAO),2024
1,2,Nigeria,Sub-Saharan Africa,2024
2,3,Indonesia,Central & Southern Asia and Oceania (CSAO),2024
3,4,United States,North America,2024
4,5,Vietnam,Central & Southern Asia and Oceania (CSAO),2024


In [24]:
# Create a country → region mapping from 2023
region_map_2023 = df_2023.set_index("Country")["Region"].to_dict()

# If Region column is missing entirely, create it first
if "Region" not in df_2022.columns:
    df_2022["Region"] = None

# Fill region using the map
df_2022["Region"] = df_2022["Country"].map(region_map_2023)

missing_regions = df_2022[df_2022["Region"].isna()]
print("Countries without matched region:", missing_regions["Country"].tolist())

Countries without matched region: ['Palestine', 'Côte d’Ivoire', 'Belize', 'Democratic Republic of the Congo', 'Macedonia', 'Saint Lucia', 'Congo', 'Bermuda']


In [25]:
# Drop rows with missing Region
df_2022 = df_2022.dropna(subset=["Region"])
missing_regions = df_2022[df_2022["Region"].isna()]
print("Countries without matched region:", missing_regions["Country"].tolist())

Countries without matched region: []


In [26]:
# Combine into one DataFrame
GCAI_df = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

# Show result
GCAI_df

,Rank,Country,Year,Region
0,1,Vietnam,2022,Central & Southern Asia and Oceania
1,2,Philippines,2022,Central & Southern Asia and Oceania
2,3,Ukraine,2022,Eastern Europe
3,4,India,2022,Central & Southern Asia and Oceania
4,5,United States,2022,North America
...,...,...,...,...
439,147,Mauritius,2024,Sub-Saharan Africa
440,148,Belize,2024,Latin America (LATAM)
441,149,Rep. of Congo,2024,Sub-Saharan Africa
442,150,Mali,2024,Sub-Saharan Africa


In [28]:
# List unique regions for each year
regions_2022 = df_2022["Region"].dropna().unique()
regions_2023 = df_2023["Region"].dropna().unique()
regions_2024 = df_2024["Region"].dropna().unique()

# Display them
print("2022:", sorted(regions_2022))
print("2023:", sorted(regions_2023))
print("2024:", sorted(regions_2024))

2022: ['Central & Southern Asia and Oceania', 'Central Northern & Western Europe', 'Eastern Asia', 'Eastern Europe', 'Latin America', 'Middle East & North Africa', 'North America', 'Sub-Saharan Africa']
2023: ['Central & Southern Asia and Oceania', 'Central Northern & Western Europe', 'Eastern Asia', 'Eastern Europe', 'Latin America', 'Middle East & North Africa', 'North America', 'Sub-Saharan Africa']
2024: ['Central & Southern Asia and Oceania (CSAO)', 'Central Northern & Western Europe (CNWE)', 'Eastern Asia', 'Eastern Europe', 'Latin America (LATAM)', 'Middle East & North Africa (MENA)', 'North America', 'Sub-Saharan Africa']


## Match 2024 region as 2023 and 2022

In [30]:
# Fix Region names ONLY for rows where Year == 2024
GCAI_df.loc[GCAI_df["Year"] == 2024, "Region"] = GCAI_df.loc[
    GCAI_df["Year"] == 2024, "Region"
].str.replace(r"\s+\(.*\)", "", regex=True)
# Print unique regions per year
print("2022:", sorted(GCAI_df[GCAI_df["Year"] == 2022]["Region"].dropna().unique()))
print("2023:", sorted(GCAI_df[GCAI_df["Year"] == 2023]["Region"].dropna().unique()))
print("2024:", sorted(GCAI_df[GCAI_df["Year"] == 2024]["Region"].dropna().unique()))

2022: ['Central & Southern Asia and Oceania', 'Central Northern & Western Europe', 'Eastern Asia', 'Eastern Europe', 'Latin America', 'Middle East & North Africa', 'North America', 'Sub-Saharan Africa']
2023: ['Central & Southern Asia and Oceania', 'Central Northern & Western Europe', 'Eastern Asia', 'Eastern Europe', 'Latin America', 'Middle East & North Africa', 'North America', 'Sub-Saharan Africa']
2024: ['Central & Southern Asia and Oceania', 'Central Northern & Western Europe', 'Eastern Asia', 'Eastern Europe', 'Latin America', 'Middle East & North Africa', 'North America', 'Sub-Saharan Africa']


## 2. Importing International Migrant Stock Data (2024)

We will import the Excel file containing international migrant stock data by sex, destination, and origin for the year 2024.

**Source:** [undesa_pd_2024_ims_stock_by_sex_destination_and_origin.raw.xlsx](https://github.com/MIT-Emerging-Talent/ET6-CDSP-group-12-repo/blob/main/1_datasets/raw_datasets_files/undesa_pd_2024_ims_stock_by_sex_destination_and_origin.raw.xlsx)


In [43]:
# Raw URL of the Excel file
migrant_stock_url = "https://raw.githubusercontent.com/MIT-Emerging-Talent/ET6-CDSP-group-12-repo/main/1_datasets/raw_datasets_files/undesa_pd_2024_ims_stock_by_sex_destination_and_origin.raw.xlsx"

# Load the Excel file and inspect sheet names
migrant_stock_raw_df = pd.read_excel(migrant_stock_url, sheet_name="Table 2")

## Extract and match the region of Migrant Stock with Global Crypto Adoptation Index

In [44]:
# Slice rows 8 to 62 (0-based index: 7 to 61), and columns A to BE (0 to 56)
excel_rows = [13, 38, 39, 40, 44, 45, 47, 50, 51, 52, 53, 54, 58, 59]
row_indices = [i - 2 for i in excel_rows]

# Columns B and E → indices 1 and 4
migrant_stock_df = migrant_stock_raw_df.iloc[row_indices, [1, 4]]

# Optional: Reset index and set column names
migrant_stock_df.columns = ["Region", "2024 Migrant Stock"]
migrant_stock_df = migrant_stock_df.reset_index(drop=True)
# Display the cleaned DataFrame
print(migrant_stock_df)

                              Region 2024 Migrant Stock
0                 Sub-Saharan Africa           24306400
1                     Eastern Africa            8302733
2                      Middle Africa            4854994
3                    Northern Africa            4874040
4                       Central Asia            3910116
5                       Eastern Asia           11559387
6                      Southern Asia           16459536
7                     Eastern Europe           18646093
8                    Northern Europe           18543047
9                    Southern Europe           20119984
10                    Western Europe           36742379
11   LATIN AMERICA AND THE CARIBBEAN           17510527
12                  NORTHERN AMERICA           61207877
13                          OCEANIA             9901164
